In [18]:
from sklearn.datasets import load_iris
import numpy as np
from torch import tensor , nn
import torch
import matplotlib.pyplot as plt

In [19]:
data = load_iris()

X=tensor(data.data[:,[1,3]], dtype=torch.float)
y = tensor(data.target, dtype=torch.int)

In [20]:
torch.manual_seed(123)
shuffle_idx = torch.randperm(y.size(0), dtype=torch.long)

X, y = X[shuffle_idx], y[shuffle_idx]

percent80 = int(shuffle_idx.size(0)*0.8)

X_train, X_test = X[shuffle_idx[:percent80]], X[shuffle_idx[percent80:]]
y_train, y_test = y[shuffle_idx[:percent80]], y[shuffle_idx[percent80:]]

In [21]:
class logreg_model (nn.Module):
    def __init__(self) -> None:
        super().__init__()

        self.linear = nn.Linear(in_features=2, out_features=3)

        self.linear.weight.detach().zero_()

        self.linear.bias.detach().zero_()

    def forward(self, x):

        self.regression = self.linear(x)

        self.probability = nn.functional.softmax(self.regression, dim=1)

        return self.regression, self.probability

In [22]:
model = logreg_model()

print(model.parameters)

<bound method Module.parameters of logreg_model(
  (linear): Linear(in_features=2, out_features=3, bias=True)
)>


In [23]:
def comp_accuracy(true_labels, pred_labels):
    accuracy = torch.sum(true_labels.view(-1).float() == 
                         pred_labels.float()).item() / true_labels.size(0)
    return accuracy

In [24]:
optimiser = torch.optim.SGD(model.parameters(),lr = 0.1)

In [25]:
epochs = 300

for i in range(epochs):

    reg_results , prob_results = model(X_train)

    loss = nn.CrossEntropyLoss()

    output = loss(reg_results, y_train.long())

    acc=comp_accuracy(y_train, torch.argmax(prob_results, dim=1))

    optimiser.zero_grad()

    output.backward()

    optimiser.step()

    test_reg_results, test_prob_results =model(X_test)

    test_acc=comp_accuracy(y_test, torch.argmax(test_prob_results, dim=1))
    
    if i % 10 ==0:

        print(f"Epoch: {i} | Loss: {output:.5f}, Train accuracy: {acc *100:.2f}% | Test accuracy: {test_acc*100:.2f}%")

Epoch: 0 | Loss: 1.09861, Train accuracy: 34.17% | Test accuracy: 56.67%
Epoch: 10 | Loss: 0.92625, Train accuracy: 69.17% | Test accuracy: 56.67%
Epoch: 20 | Loss: 0.81701, Train accuracy: 69.17% | Test accuracy: 56.67%
Epoch: 30 | Loss: 0.74180, Train accuracy: 69.17% | Test accuracy: 56.67%
Epoch: 40 | Loss: 0.68718, Train accuracy: 69.17% | Test accuracy: 56.67%
Epoch: 50 | Loss: 0.64560, Train accuracy: 70.00% | Test accuracy: 60.00%
Epoch: 60 | Loss: 0.61268, Train accuracy: 71.67% | Test accuracy: 60.00%
Epoch: 70 | Loss: 0.58579, Train accuracy: 73.33% | Test accuracy: 60.00%
Epoch: 80 | Loss: 0.56325, Train accuracy: 76.67% | Test accuracy: 60.00%
Epoch: 90 | Loss: 0.54396, Train accuracy: 80.00% | Test accuracy: 60.00%
Epoch: 100 | Loss: 0.52716, Train accuracy: 84.17% | Test accuracy: 70.00%
Epoch: 110 | Loss: 0.51231, Train accuracy: 87.50% | Test accuracy: 70.00%
Epoch: 120 | Loss: 0.49904, Train accuracy: 88.33% | Test accuracy: 76.67%
Epoch: 130 | Loss: 0.48704, Train ac